# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/keegan/spark-2.2.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('churn').getOrCreate()

In [14]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [15]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [16]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [17]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) for column in list(set(data.columns)-set(['Names','Age','Total_Purchase',"Account_Manager","Years","Num_Sites","Onboard_date"])) ]
#indexer = StringIndexer(inputCol="Location", outputCol="loc")
#indexer2 = StringIndexer(inputCol="Company", outputCol="com")

In [18]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Num_Sites',
 'Years',
  'Account_Manager'                                     
                                    
 ],outputCol='features')

In [19]:
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(data).transform(data)
#output = assembler.transform(indexers)

In [20]:
df_r.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------+-------------+-----------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|Location_index|Company_index|Churn_index|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------+-------------+-----------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|         709.0|        824.0|        1.0|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|         710.0|          1.0|        1.0|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 0

In [21]:
output = assembler.transform(df_r)

In [22]:
final_data=output.select("features",'churn')

In [23]:
from pyspark.ml.classification import LogisticRegression

In [34]:
train_output, test_putput = final_data.randomSplit([0.7,.3])

In [35]:
train_output.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[22.0,11254.38,8....|    0|
|[26.0,8787.39,11....|    1|
|[26.0,8939.61,7.0...|    0|
|[28.0,8670.98,6.0...|    0|
|[28.0,11128.95,8....|    0|
|[28.0,11204.23,11...|    0|
|[29.0,5900.78,8.0...|    0|
|[29.0,8688.17,9.0...|    1|
|[29.0,9378.24,8.0...|    0|
|[29.0,9617.59,8.0...|    0|
|[29.0,12711.15,7....|    0|
|[29.0,13255.05,8....|    0|
|[30.0,6744.87,9.0...|    0|
|[30.0,7960.64,10....|    1|
|[30.0,8403.78,7.0...|    0|
|[30.0,8677.28,7.0...|    0|
|[30.0,8874.83,9.0...|    0|
|[30.0,10183.98,9....|    0|
|[30.0,10960.52,9....|    0|
|[30.0,12788.37,10...|    0|
+--------------------+-----+
only showing top 20 rows



In [36]:
log_reg_output = LogisticRegression(featuresCol='features',labelCol='churn')

In [37]:
logModel = log_reg_output.fit(train_output)

In [38]:
results = logModel.transform(test_putput)

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='churn')

In [42]:
results.select('churn','prediction').show()

+-----+----------+
|churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [43]:
AUC = my_eval.evaluate(results)

In [44]:
AUC

0.7655349135169763